In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
from lib import helper

In [3]:
mnist = input_data.read_data_sets("data/", one_hot=True)
test_data = mnist.test.images
test_label = mnist.test.labels

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz


In [4]:
num_inputs = 784 # 28x28
num_outputs = 10
num_units = 128
num_layers = 8
batch_size = 64
num_epocs= 200000
learning_rate = 0.001
use_dropout = True
in_dropout_rate = 0.05
all_dropout_rate = 0.1

In [5]:
myint = tf.int32
myfloat = tf.float32

In [6]:
# placeholder
x = tf.placeholder(dtype=myfloat, shape=[None, num_inputs], name='inputs')
y = tf.placeholder(dtype=myfloat, shape=[None, num_outputs], name='outputs')
is_training = tf.placeholder(dtype=tf.bool, name='is_training')

In [7]:
# network
if use_dropout:
    x = tf.layers.dropout(x, rate=in_dropout_rate, training=is_training)
for i in range(num_layers):
    if i ==0:
        layer = tf.layers.dense(x, num_units, tf.nn.relu, kernel_initializer=tf.truncated_normal_initializer(stddev=0.1), name='l_{}'.format(i))
    else:
        layer = tf.layers.dense(layer, num_units, tf.nn.relu, kernel_initializer=tf.truncated_normal_initializer(stddev=0.1), name='l_{}'.format(i))
    if use_dropout:
        layer = tf.layers.dropout(layer, rate=all_dropout_rate , training=is_training, name='l_dropout_{}'.format(i))

out = tf.layers.dense(layer,
                           num_outputs, 
                           tf.nn.softmax, 
                           kernel_initializer=tf.truncated_normal_initializer(stddev=0.1), 
                           name='out_layer')

In [8]:
loss_ce = tf.reduce_mean(-tf.reduce_sum(y * tf.log(out + 10e-8), 1))
loss_se = tf.reduce_sum(tf.square(y - out))

optimizer = tf.train.AdamOptimizer(learning_rate)
params = tf.trainable_variables()
gradients = tf.gradients(loss_ce, params)
train = optimizer.apply_gradients(zip(gradients, params))

grad_norm = [tf.norm(g) for g in gradients]
grad_norm_sum = tf.reduce_sum(grad_norm)

In [9]:
y_label = tf.argmax(y, 1)
out_label = tf.argmax(out, 1)

correct = tf.equal(y_label, out_label)
accuracy = tf.reduce_mean(tf.cast(correct, myfloat))

In [10]:
with tf.name_scope('train'):
    smr_loss_ce = tf.summary.scalar('loss_cross_entropy', loss_ce)
    smr_loss_se = tf.summary.scalar('loss_squared_error', loss_se)
    smr_acc = tf.summary.scalar('accuracy', accuracy)
    smr_grad = tf.summary.scalar('gradient', grad_norm_sum)
    merged_summary = tf.summary.merge([smr_loss_ce, smr_loss_se, smr_acc, smr_grad])

with tf.name_scope('test'):
    test_smr_acc = tf.summary.scalar('accuracy', accuracy)

init = tf.global_variables_initializer()

In [11]:
from datetime import datetime
now = datetime.now()
use_dropout_str = 'on' if use_dropout else 'off'
logdir_base = 'logs/dropout_relu/dropout={}/units={}/layers={}/2/'.format(use_dropout_str, num_units, num_layers)
logdir = logdir_base #+ now.strftime("%Y%m%d-%H%M%S") + "/"

In [12]:
logdir_base = 'logs/dropout_relu/dropout={}/units={}/layers={}/'.format(use_dropout_str, num_units, num_layers)
logdir = logdir_base #+ now.strftime("%Y%m%d-%H%M%S") + "/"
with tf.Session() as sess:
    writer = tf.summary.FileWriter(logdir, sess.graph)
    sess.run(init)
    for i in range(num_epocs):
        train_data, train_label = mnist.train.next_batch(batch_size)
        train_step, loss, train_acc, smr = sess.run([train, loss_se, accuracy, merged_summary], feed_dict={
            x: train_data,
            y: train_label,
            is_training: True
        })
        writer.add_summary(smr, i)

        if i % 10 == 0:
            
            test_acc_1, test_smr = sess.run([accuracy, test_smr_acc], feed_dict={
                x: test_data,
                y: test_label,
                is_training: False
            })
            print('epoc: ', i, ', loss: ', loss, ', train_acc: ', train_acc, 'test_acc, ', test_acc_1)
            writer.add_summary(test_smr, i)

    test_acc = sess.run(accuracy, feed_dict={
                x: test_data,
                y: test_label,
                is_training: False
            })
    print('test accuracy: ', test_acc)
    writer.close()

epoc:  0 , loss:  57.6515 , train_acc:  0.09375 test_acc,  0.1165
epoc:  10 , loss:  57.2073 , train_acc:  0.125 test_acc,  0.2644
epoc:  20 , loss:  54.942 , train_acc:  0.28125 test_acc,  0.3931
epoc:  30 , loss:  44.897 , train_acc:  0.4375 test_acc,  0.5138
epoc:  40 , loss:  40.6778 , train_acc:  0.4375 test_acc,  0.6042
epoc:  50 , loss:  29.8053 , train_acc:  0.71875 test_acc,  0.6887
epoc:  60 , loss:  29.846 , train_acc:  0.6875 test_acc,  0.7297
epoc:  70 , loss:  31.0125 , train_acc:  0.65625 test_acc,  0.7664
epoc:  80 , loss:  20.7908 , train_acc:  0.765625 test_acc,  0.8178
epoc:  90 , loss:  18.406 , train_acc:  0.8125 test_acc,  0.8298
epoc:  100 , loss:  18.1866 , train_acc:  0.828125 test_acc,  0.8157
epoc:  110 , loss:  24.8176 , train_acc:  0.671875 test_acc,  0.805
epoc:  120 , loss:  16.3227 , train_acc:  0.875 test_acc,  0.8273
epoc:  130 , loss:  26.3221 , train_acc:  0.6875 test_acc,  0.8405
epoc:  140 , loss:  18.8679 , train_acc:  0.78125 test_acc,  0.8622
ep

epoc:  1230 , loss:  7.70425 , train_acc:  0.9375 test_acc,  0.9565
epoc:  1240 , loss:  5.89148 , train_acc:  0.953125 test_acc,  0.9542
epoc:  1250 , loss:  5.68533 , train_acc:  0.953125 test_acc,  0.9531
epoc:  1260 , loss:  4.38045 , train_acc:  0.96875 test_acc,  0.9552
epoc:  1270 , loss:  5.71403 , train_acc:  0.96875 test_acc,  0.957
epoc:  1280 , loss:  8.06414 , train_acc:  0.921875 test_acc,  0.9539
epoc:  1290 , loss:  8.09226 , train_acc:  0.90625 test_acc,  0.9595
epoc:  1300 , loss:  6.60634 , train_acc:  0.9375 test_acc,  0.9604
epoc:  1310 , loss:  3.37526 , train_acc:  0.984375 test_acc,  0.9616
epoc:  1320 , loss:  6.19759 , train_acc:  0.9375 test_acc,  0.9598
epoc:  1330 , loss:  3.85602 , train_acc:  0.953125 test_acc,  0.9551
epoc:  1340 , loss:  2.80277 , train_acc:  0.984375 test_acc,  0.9588
epoc:  1350 , loss:  3.09741 , train_acc:  0.96875 test_acc,  0.9598
epoc:  1360 , loss:  3.68072 , train_acc:  0.96875 test_acc,  0.961
epoc:  1370 , loss:  5.78947 , tr

epoc:  2450 , loss:  3.5469 , train_acc:  0.96875 test_acc,  0.9633
epoc:  2460 , loss:  3.59481 , train_acc:  0.96875 test_acc,  0.9662
epoc:  2470 , loss:  5.40954 , train_acc:  0.9375 test_acc,  0.9656
epoc:  2480 , loss:  1.42276 , train_acc:  0.984375 test_acc,  0.966
epoc:  2490 , loss:  2.23712 , train_acc:  0.984375 test_acc,  0.9686
epoc:  2500 , loss:  1.63679 , train_acc:  0.984375 test_acc,  0.9683
epoc:  2510 , loss:  4.86853 , train_acc:  0.953125 test_acc,  0.9672
epoc:  2520 , loss:  3.88428 , train_acc:  0.953125 test_acc,  0.9701
epoc:  2530 , loss:  6.12389 , train_acc:  0.953125 test_acc,  0.9646
epoc:  2540 , loss:  5.7957 , train_acc:  0.9375 test_acc,  0.9651
epoc:  2550 , loss:  1.09063 , train_acc:  1.0 test_acc,  0.9673
epoc:  2560 , loss:  8.37349 , train_acc:  0.90625 test_acc,  0.9689
epoc:  2570 , loss:  6.3833 , train_acc:  0.9375 test_acc,  0.9678
epoc:  2580 , loss:  3.0131 , train_acc:  0.96875 test_acc,  0.968
epoc:  2590 , loss:  3.67161 , train_acc:

epoc:  3670 , loss:  3.14573 , train_acc:  0.96875 test_acc,  0.9682
epoc:  3680 , loss:  4.35722 , train_acc:  0.953125 test_acc,  0.9675
epoc:  3690 , loss:  0.585359 , train_acc:  1.0 test_acc,  0.968
epoc:  3700 , loss:  2.63442 , train_acc:  0.96875 test_acc,  0.969
epoc:  3710 , loss:  6.14476 , train_acc:  0.953125 test_acc,  0.9664
epoc:  3720 , loss:  3.93928 , train_acc:  0.9375 test_acc,  0.9661
epoc:  3730 , loss:  2.89465 , train_acc:  0.953125 test_acc,  0.9664
epoc:  3740 , loss:  5.04076 , train_acc:  0.9375 test_acc,  0.9686
epoc:  3750 , loss:  4.57041 , train_acc:  0.953125 test_acc,  0.9667
epoc:  3760 , loss:  3.61393 , train_acc:  0.96875 test_acc,  0.9678
epoc:  3770 , loss:  0.404342 , train_acc:  1.0 test_acc,  0.9675
epoc:  3780 , loss:  2.127 , train_acc:  0.984375 test_acc,  0.9674
epoc:  3790 , loss:  2.11855 , train_acc:  0.96875 test_acc,  0.9696
epoc:  3800 , loss:  1.11572 , train_acc:  1.0 test_acc,  0.966
epoc:  3810 , loss:  0.187441 , train_acc:  1.

epoc:  4920 , loss:  2.84354 , train_acc:  0.96875 test_acc,  0.9708
epoc:  4930 , loss:  3.65334 , train_acc:  0.953125 test_acc,  0.975
epoc:  4940 , loss:  2.97146 , train_acc:  0.96875 test_acc,  0.9741
epoc:  4950 , loss:  0.926288 , train_acc:  0.984375 test_acc,  0.9723
epoc:  4960 , loss:  1.4354 , train_acc:  0.984375 test_acc,  0.9753
epoc:  4970 , loss:  0.46976 , train_acc:  1.0 test_acc,  0.9755
epoc:  4980 , loss:  3.01837 , train_acc:  0.96875 test_acc,  0.9731
epoc:  4990 , loss:  2.1154 , train_acc:  0.984375 test_acc,  0.9729
epoc:  5000 , loss:  4.05078 , train_acc:  0.953125 test_acc,  0.9745
epoc:  5010 , loss:  1.50453 , train_acc:  0.984375 test_acc,  0.9712
epoc:  5020 , loss:  4.02111 , train_acc:  0.96875 test_acc,  0.9724
epoc:  5030 , loss:  2.03343 , train_acc:  0.984375 test_acc,  0.9725
epoc:  5040 , loss:  2.25199 , train_acc:  0.984375 test_acc,  0.9744
epoc:  5050 , loss:  2.7878 , train_acc:  0.953125 test_acc,  0.9733
epoc:  5060 , loss:  4.4559 , tr

...

epoc:  194470 , loss:  0.000282468 , train_acc:  1.0 test_acc,  0.9804
epoc:  194480 , loss:  1.74657 , train_acc:  0.984375 test_acc,  0.98
epoc:  194490 , loss:  0.000110011 , train_acc:  1.0 test_acc,  0.98
epoc:  194500 , loss:  0.111039 , train_acc:  1.0 test_acc,  0.9804
epoc:  194510 , loss:  0.00193524 , train_acc:  1.0 test_acc,  0.9805
epoc:  194520 , loss:  0.000500362 , train_acc:  1.0 test_acc,  0.9808
epoc:  194530 , loss:  1.06889e-10 , train_acc:  1.0 test_acc,  0.9805
epoc:  194540 , loss:  1.52048e-05 , train_acc:  1.0 test_acc,  0.9806
epoc:  194550 , loss:  0.000468737 , train_acc:  1.0 test_acc,  0.9807
epoc:  194560 , loss:  0.00457307 , train_acc:  1.0 test_acc,  0.9809
epoc:  194570 , loss:  1.59996 , train_acc:  0.984375 test_acc,  0.981
epoc:  194580 , loss:  0.444578 , train_acc:  1.0 test_acc,  0.9808
epoc:  194590 , loss:  1.11101e-06 , train_acc:  1.0 test_acc,  0.9813
epoc:  194600 , loss:  1.31617e-07 , train_acc:  1.0 test_acc,  0.9816
epoc:  194610 , l

epoc:  195650 , loss:  3.14231e-05 , train_acc:  1.0 test_acc,  0.9805
epoc:  195660 , loss:  6.96127e-06 , train_acc:  1.0 test_acc,  0.9805
epoc:  195670 , loss:  1.46938e-07 , train_acc:  1.0 test_acc,  0.9799
epoc:  195680 , loss:  0.00794524 , train_acc:  1.0 test_acc,  0.9802
epoc:  195690 , loss:  0.0314456 , train_acc:  1.0 test_acc,  0.9811
epoc:  195700 , loss:  4.37583 , train_acc:  0.953125 test_acc,  0.9796
epoc:  195710 , loss:  0.00630733 , train_acc:  1.0 test_acc,  0.98
epoc:  195720 , loss:  3.32721e-09 , train_acc:  1.0 test_acc,  0.9805
epoc:  195730 , loss:  0.0152715 , train_acc:  1.0 test_acc,  0.9808
epoc:  195740 , loss:  0.0142908 , train_acc:  1.0 test_acc,  0.9804
epoc:  195750 , loss:  1.18427e-09 , train_acc:  1.0 test_acc,  0.9813
epoc:  195760 , loss:  4.62377e-06 , train_acc:  1.0 test_acc,  0.9809
epoc:  195770 , loss:  0.0477827 , train_acc:  1.0 test_acc,  0.9811
epoc:  195780 , loss:  0.00856689 , train_acc:  1.0 test_acc,  0.9823
epoc:  195790 , lo

epoc:  196830 , loss:  7.30027e-05 , train_acc:  1.0 test_acc,  0.9803
epoc:  196840 , loss:  0.320405 , train_acc:  1.0 test_acc,  0.9804
epoc:  196850 , loss:  4.01097e-05 , train_acc:  1.0 test_acc,  0.9797
epoc:  196860 , loss:  1.21613e-11 , train_acc:  1.0 test_acc,  0.9794
epoc:  196870 , loss:  0.000660636 , train_acc:  1.0 test_acc,  0.9799
epoc:  196880 , loss:  0.00280172 , train_acc:  1.0 test_acc,  0.9797
epoc:  196890 , loss:  0.76647 , train_acc:  0.984375 test_acc,  0.9798
epoc:  196900 , loss:  0.00012322 , train_acc:  1.0 test_acc,  0.98
epoc:  196910 , loss:  0.0241984 , train_acc:  1.0 test_acc,  0.9793
epoc:  196920 , loss:  0.403203 , train_acc:  1.0 test_acc,  0.9794
epoc:  196930 , loss:  1.78634 , train_acc:  0.984375 test_acc,  0.9794
epoc:  196940 , loss:  1.99196 , train_acc:  0.984375 test_acc,  0.9801
epoc:  196950 , loss:  0.217906 , train_acc:  1.0 test_acc,  0.9802
epoc:  196960 , loss:  2.19854e-10 , train_acc:  1.0 test_acc,  0.9795
epoc:  196970 , lo

epoc:  198040 , loss:  0.00115832 , train_acc:  1.0 test_acc,  0.9804
epoc:  198050 , loss:  1.14784e-06 , train_acc:  1.0 test_acc,  0.9799
epoc:  198060 , loss:  0.410784 , train_acc:  1.0 test_acc,  0.9794
epoc:  198070 , loss:  2.00011 , train_acc:  0.984375 test_acc,  0.9799
epoc:  198080 , loss:  1.4287 , train_acc:  0.984375 test_acc,  0.9812
epoc:  198090 , loss:  0.00974889 , train_acc:  1.0 test_acc,  0.9807
epoc:  198100 , loss:  0.273166 , train_acc:  1.0 test_acc,  0.9808
epoc:  198110 , loss:  2.2191e-05 , train_acc:  1.0 test_acc,  0.9808
epoc:  198120 , loss:  0.214851 , train_acc:  1.0 test_acc,  0.9804
epoc:  198130 , loss:  3.01026e-06 , train_acc:  1.0 test_acc,  0.9803
epoc:  198140 , loss:  0.00588112 , train_acc:  1.0 test_acc,  0.9804
epoc:  198150 , loss:  0.00180133 , train_acc:  1.0 test_acc,  0.9804
epoc:  198160 , loss:  1.08574e-12 , train_acc:  1.0 test_acc,  0.98
epoc:  198170 , loss:  0.00618133 , train_acc:  1.0 test_acc,  0.9802
epoc:  198180 , loss: 

epoc:  199230 , loss:  0.700579 , train_acc:  1.0 test_acc,  0.9822
epoc:  199240 , loss:  1.99842 , train_acc:  0.984375 test_acc,  0.9819
epoc:  199250 , loss:  0.0018715 , train_acc:  1.0 test_acc,  0.9821
epoc:  199260 , loss:  1.00876 , train_acc:  0.984375 test_acc,  0.982
epoc:  199270 , loss:  0.15227 , train_acc:  1.0 test_acc,  0.9819
epoc:  199280 , loss:  2.32714 , train_acc:  0.96875 test_acc,  0.9812
epoc:  199290 , loss:  0.033459 , train_acc:  1.0 test_acc,  0.9801
epoc:  199300 , loss:  4.79001 , train_acc:  0.953125 test_acc,  0.9806
epoc:  199310 , loss:  1.57861 , train_acc:  0.984375 test_acc,  0.9809
epoc:  199320 , loss:  0.00680473 , train_acc:  1.0 test_acc,  0.9806
epoc:  199330 , loss:  0.412491 , train_acc:  1.0 test_acc,  0.9805
epoc:  199340 , loss:  1.93971 , train_acc:  0.984375 test_acc,  0.9803
epoc:  199350 , loss:  0.0889862 , train_acc:  1.0 test_acc,  0.9801
epoc:  199360 , loss:  0.0717899 , train_acc:  1.0 test_acc,  0.9798
epoc:  199370 , loss: 